# Loading Necessary Packages

In [ ]:
import numpy as np
import pandas as pd
import json
import gensim
import re
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import dot
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.preprocessing.sequence import skipgrams, make_sampling_table
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras_preprocessing.text import Tokenizer

In [ ]:
import requests
from gensim.models import Word2Vec
from keras import Input, Model
from keras.activations import softmax
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences

from keras_preprocessing.text import Tokenizer
import pandas as pd




# Loading Fine-Tune Dataset

In [ ]:
df_finetune = pd.read_csv("/Users/shivanibalaji/Downloads/faq.csv")
df_finetune.head()

,Question,Answer,Unnamed: 2,Unnamed: 3
0,Are session pre-class quizzes graded?,No. Pre-class quiz is to just check your under...,NaN,NaN
1,When is the deadline for the submission of ses...,5 PM on the day of the following lecture.,NaN,NaN
2,Will there be any extension allowed for the qu...,Only for exceptional cases.,NaN,NaN
3,How many hours will I need to dedicate to succ...,About 15 hours per week.,NaN,NaN
4,Who will grade my exercise?,The exercises are auto-graded once you click t...,NaN,NaN


In [ ]:
len(df_finetune)

33

# Train Dataset - SQUaD 

#  SQUAD Dataset Preprocessing

In [ ]:
train = pd.read_json("train-v2.0.json")

### Json to Dataframe

In [ ]:
title = []
context = []
questions = []
answers_text = []
answers_start = []
id_num = []
is_impossible_list = []
plausible_answers_list = []
plausible_answers_start_list= []

for i in range(train.shape[0]):      # all titles(442)
    title_name = train.data[i]['title']     

    for j in range(len(train.data[i]['paragraphs'])): #number of paragraphs in title
        context_name = train.data[i]['paragraphs'][j]['context']
        
        for k in range(len(train.data[i]['paragraphs'][j]['qas'])): #questions for each paragraph

            question_name =  train.data[i]['paragraphs'][j]['qas'][k]['question']
            id_number =    train.data[i]['paragraphs'][j]['qas'][k]['id']
            
            is_impossible = train.data[i]['paragraphs'][j]['qas'][k]['is_impossible']
            
            if is_impossible == True:
                p_answers_text_name = train.data[i]['paragraphs'][j]['qas'][k]['plausible_answers'][0]['text']
                p_answers_text_start = train.data[i]['paragraphs'][j]['qas'][k]['plausible_answers'][0]['answer_start']
                
            elif is_impossible == False:
                p_answers_text_name = 'n/a'
                p_answers_text_start = 'n/a'
                
            
            
            if train.data[i]['paragraphs'][j]['qas'][k]['answers'] != []:
                answers_text_name = train.data[i]['paragraphs'][j]['qas'][k]['answers'][0]['text']
                answers_start_num = train.data[i]['paragraphs'][j]['qas'][k]['answers'][0]['answer_start']
            elif train.data[i]['paragraphs'][j]['qas'][k]['answers'] == []:
                answers_text_name = 'n/a'
                answers_start_num = 'n/a'
            

            title.append(title_name)
            context.append(context_name)
            questions.append(question_name)
            answers_text.append(answers_text_name)
            answers_start.append(answers_start_num)
            id_num.append(id_number)
            is_impossible_list.append(is_impossible)
            plausible_answers_list.append(p_answers_text_name)
            plausible_answers_start_list.append(p_answers_text_start)
            

In [ ]:
plausible_answers_list[130314]

'matter'

In [ ]:
train.data[441]['paragraphs'][1]['qas'][0]['plausible_answers'][0]['text']

'a cloud'

In [ ]:
squad = pd.DataFrame({"title":title, "context": context, "questions": questions, "answers_text": answers_text,"answers_start":answers_start, "id_num":id_num, "is_impossible":is_impossible_list, "plausible_answer":plausible_answers_list, "plausible_answer_start":plausible_answers_start_list  })

In [ ]:
squad.tail()

,title,context,questions,answers_text,answers_start,id_num,is_impossible,plausible_answer,plausible_answer_start
130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,n/a,n/a,5a7e070b70df9f001a875439,True,matter,485
130315,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,n/a,n/a,5a7e070b70df9f001a87543a,True,Alfvén,327
130316,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,n/a,n/a,5a7e070b70df9f001a87543b,True,Gk. common matter,350
130317,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,n/a,n/a,5a7e070b70df9f001a87543c,True,a specifying modifier,529
130318,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,n/a,n/a,5a7e070b70df9f001a87543d,True,physics,37


In [ ]:
squad.to_csv(r'Downloads\SQUaD4BERT.csv', index = False)

In [ ]:
squad_small= squad[:10000]

In [ ]:
squad_small.to_csv(r'Downloads\SQUaD4seq2seq.csv', index = False)

### Removing Impossible Values

In [ ]:
squad['is_impossible'].value_counts()

False    86821
True     43498
Name: is_impossible, dtype: int64

In [ ]:
squad_new = squad[['questions', 'answers_text']]
squad_new = squad_new[squad_new.answers_text != 'n/a']
squad_new = squad_new.reset_index(drop=True)

### Subsetting Data (due computational constraints)

In [ ]:
squad_new = squad_new[:20000]

In [ ]:
squad_new.tail()

,questions,answers_text
19995,The 1080i30 or 1080i60 notion identifies inter...,30
19996,The 720p60 notion identifies progressive scann...,60
19997,What three scanning rates do 50 Hz systems sup...,"50i, 25p and 50p"
19998,"Which system suports 59.94i, 60i, 23.976p, 24p...",60 Hz
19999,SD television fractional rates were often roun...,whole numbers


# Data Cleaning

### Defining Inputs & Targets for Train, Fine-Tune and Test

In [ ]:
questions_train = [quest for quest in squad_new.questions]
answers_train = [ans for ans in squad_new.answers_text]

In [ ]:
questions_ft = [quest for quest in df_finetune.Question]
answers_ft = [ans for ans in df_finetune.Answer]

### Cleaning + Lowercase

In [ ]:
def clean_data(sentence_list):
    
    clean_sent = [re.sub("[^a-zA-Z0-9 ]", "", i) for i in sentence_list]
    clean_sent = [word.lower() for word in clean_sent]

    return clean_sent

In [ ]:
clean_question_train = clean_data(questions_train)
clean_answer_train = clean_data(answers_train)

In [ ]:
clean_question_ft = clean_data(questions_ft)
clean_answer_ft = clean_data(answers_ft)

### Lematization + stop words removal

In [ ]:
#stopwords_set = set(stopwords.words('english'))

In [ ]:
#lemmatizer = WordNetLemmatizer()

In [ ]:
def lemma_stop_words(sentence_list):
    lemma_sent = [word_tokenize(i) for i in sentence_list]
    lemma_sent = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in lemma_sent]
    lemma_sent = [[i for i in l if i not in stopwords_set] for l in lemma_sent]
    lemma_sent = [(" ").join(sent) for sent in lemma_sent]
    
    return lemma_sent

In [ ]:
#clean_question_train = lemma_stop_words(clean_question_train)
#clean_answer_train = lemma_stop_words(clean_answer_train)

In [ ]:
#clean_question_ft = lemma_stop_words(clean_question_ft)
#clean_answer_ft = lemma_stop_words(clean_answer_ft)

In [ ]:
#clean_questions_test = lemma_stop_words(clean_questions_test)

## Tokenization

In [ ]:
from nltk.tokenize import word_tokenize 

In [ ]:
tokenized_questions_train = [word_tokenize(i) for i in clean_question_train]
tokenized_answers_train = [word_tokenize(i) for i in clean_answer_train]

tokenized_questions_ft = [word_tokenize(i) for i in clean_question_ft]
tokenized_answers_ft = [word_tokenize(i) for i in clean_answer_ft]

In [ ]:
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(clean_question_train + clean_answer_train + clean_question_ft + clean_answer_ft)

In [ ]:
#tokenized_questions_train = tokenizer.texts_to_sequences(clean_question_train)
#tokenized_answers_train = tokenizer.texts_to_sequences(clean_answer_train)

#tokenized_questions_ft = tokenizer.texts_to_sequences(clean_question_ft)
#tokenized_answers_ft = tokenizer.texts_to_sequences(clean_answer_ft)

In [ ]:
all_words_train = tokenized_questions_train + tokenized_answers_train  
all_words_ft = tokenized_questions_ft + tokenized_answers_ft

all_words_train = list(np.concatenate(all_words_train).flat)
all_words_ft = list(np.concatenate(all_words_ft).flat)

In [ ]:
unique_words_train = list(set(all_words_train))
unique_words_ft = list(set(all_words_ft))


In [ ]:
print("Unique words for train = ", len(unique_words_train))
print("Unique words for fine tune = ", len(unique_words_ft))

Unique words for train =  21629
Unique words for fine tune =  240


### Creating Vocabulary

In [ ]:
vocab= [word for word, word_count in Counter(unique_words_train).most_common(6000)]
len(vocab)

6000

In [ ]:
total_vocab = vocab + unique_words_ft
total_vocab= list(set(total_vocab))

In [ ]:
print("Vocab size = ", len(total_vocab))

Vocab size =  6186


In [ ]:
word2idx = {}
idx2word = {}

In [ ]:
i = 4
for word in total_vocab:
    if word not in word2idx:
        word2idx[word] = i
        idx2word[i] = word
        i += 1

In [ ]:
idx2word[0]= 'padding'
idx2word[1]= 'unk'
idx2word[2] = '<s>'
idx2word[3] = '</s>'

In [ ]:
word2idx['padding']= 0
word2idx['unk']= 1
word2idx['<s>'] = 2
word2idx['</s>'] = 3

In [ ]:
vocab_size = len(word2idx)

In [ ]:
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 6190


### Adding Start and End Tokens

In [ ]:
def start_end(token_sentence_list):
    for sent in token_sentence_list:
        sent.append('</s>')
        sent.insert(0,'<s>')
    return token_sentence_list

In [ ]:
tokenized_answers_ft = start_end(tokenized_answers_ft)
tokenized_answers_train = start_end(tokenized_answers_train)

### Enocde Dataset

In [ ]:
def encode(sentence_list):
    for i in range(len(sentence_list)):
        sentence = []
        
        for word in sentence_list[i]:
            
            if word in word2idx.keys():
                word = word2idx[word]
            else: 
                word = word2idx['unk']
                
            sentence.append(word)
        
        sentence_list[i] = sentence
        
    return sentence_list

In [ ]:
encoded_questions_train = encode(tokenized_questions_train)
encoded_answers_train = encode(tokenized_answers_train)

encoded_questions_ft = encode(tokenized_questions_ft)
encoded_answers_ft = encode(tokenized_answers_ft)


### Padding

In [ ]:
maxlen_questions = max(max([len(x) for x in encoded_questions_train]), max([len(x) for x in encoded_questions_ft]))
maxlen_answers = max(max([len(x) for x in encoded_answers_train]), max([len(x) for x in encoded_answers_ft]))

In [ ]:
maxlen = max(maxlen_questions, maxlen_answers)

In [ ]:
X_pad_train= pad_sequences(encoded_questions_train, maxlen=maxlen, padding='post', value = word2idx['padding'])
X_pad_ft = pad_sequences(encoded_questions_ft, maxlen=maxlen, padding='post', value = word2idx['padding'])

In [ ]:
Y_pad_train= pad_sequences(encoded_answers_train, maxlen=maxlen, padding='post', value = word2idx['padding'])
Y_pad_ft = pad_sequences(encoded_answers_ft, maxlen=maxlen, padding='post', value = word2idx['padding'])

In [ ]:
maxlen

53

# Modelling

### Base Model

Define and train a simple RNN network with 2 RNN layers as your baseline model. 

In [ ]:
Y_pad_train_m1 = [to_categorical(j, num_classes=vocab_size) for j in Y_pad_train]
Y_pad_ft_m1 = [to_categorical(j, num_classes=vocab_size) for j in Y_pad_ft]

In [ ]:
embedding_size = 10
vocab_size = len(word2idx)
maxlen = maxlen


model_baseline = Sequential()

model_baseline.add(Embedding(vocab_size, embedding_size, \
                    input_length=maxlen))

model_baseline.add(SimpleRNN(64, return_sequences = True))
model_baseline.add(SimpleRNN(64, return_sequences = True))
model_baseline.add(Dense(vocab_size, activation='softmax'))

model_baseline.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_baseline.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 53, 10)            61900     
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 53, 64)            4800      
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 53, 64)            8256      
_________________________________________________________________
dense_4 (Dense)              (None, 53, 6190)          402350    
Total params: 477,306
Trainable params: 477,306
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)

In [ ]:
#model_baseline.fit(X_pad_train, np.array(Y_pad_train_m1), epochs=50,  callbacks = callback, validation_split = 0.1)

Epoch 1/50
563/563 [==============================] - 116s 201ms/step - loss: 2.2438 - accuracy: 0.8959 - val_loss: 0.3172 - val_accuracy: 0.9439
Epoch 2/50
563/563 [==============================] - 127s 225ms/step - loss: 0.2406 - accuracy: 0.9555 - val_loss: 0.3075 - val_accuracy: 0.9432
Epoch 3/50
563/563 [==============================] - 120s 214ms/step - loss: 0.2326 - accuracy: 0.9559 - val_loss: 0.3038 - val_accuracy: 0.9440
Epoch 4/50
563/563 [==============================] - 120s 214ms/step - loss: 0.2256 - accuracy: 0.9570 - val_loss: 0.3018 - val_accuracy: 0.9448
Epoch 5/50
563/563 [==============================] - 122s 216ms/step - loss: 0.2223 - accuracy: 0.9568 - val_loss: 0.2983 - val_accuracy: 0.9447
Epoch 6/50
563/563 [==============================] - 123s 219ms/step - loss: 0.2169 - accuracy: 0.9568 - val_loss: 0.3054 - val_accuracy: 0.9451
Epoch 7/50
563/563 [==============================] - 123s 219ms/step - loss: 0.2136 - accuracy: 0.9575 - val_loss: 0.3072 -

In [ ]:


import pickle

print("Saving model...")
model_baseline.save('model_baseline')

with open('shak-nlg-dict.pkl', 'wb') as handle:
    pickle.dump(word2idx, handle)

with open('shak-nlg-maxlen.pkl', 'wb') as handle:
    pickle.dump(maxlen, handle)
print("Model Saved!") 



Saving model...
INFO:tensorflow:Assets written to: model_baseline/assets
Model Saved!


In [ ]:
#model_baseline.save_weights('model_baseline_weights.h5')

In [ ]:
#import pickle
#import keras

model_baseline = keras.models.load_model('model_baseline')
maxlen = pickle.load(open('shak-nlg-maxlen.pkl', 'rb'))
word_indexes = pickle.load(open('shak-nlg-dict.pkl', 'rb'))

### Predictions

In [ ]:
def predict(sentence, model): 
    
    #preprocessing
    clean_sent = re.sub("[^a-zA-Z0-9 ]", "", sentence)
    clean_sent = clean_sent.lower()
    
    #tokenize
    token_sent = word_tokenize(clean_sent)

    #encode
    token_sent_new = []
    
    for word in token_sent:
        if word in word2idx.keys():
            word = word2idx[word]
        else: 
            word = word2idx['unk']
        
        token_sent_new.append(word)
            
    #padding
    sent_padded = pad_sequences([token_sent_new], maxlen=maxlen, padding='post', value = word2idx['padding'])

    
    #prediction
    prediction = np.argmax(model.predict(sent_padded), axis=2)
    prediction = np.delete(prediction[0], np.where(prediction[0] == 0))
    
    #decode    
    decoded_sentence = [idx2word[p] for p in prediction]
    
    #remove start/end tags
    
    if '<s>' in decoded_sentence: 
        decoded_sentence.remove('<s>')
    if '</s>' in decoded_sentence: 
        decoded_sentence.remove('</s>')
    
    #final sentence
    predicted_sentence = (" ").join(decoded_sentence)
    
    
    return predicted_sentence

In [ ]:
predict('This is Beyonce', model_baseline)

'unk unk'

## Fine Tuning

In [ ]:
import keras

In [ ]:
embedding_size = 10
vocab_size = len(word2idx)
maxlen = maxlen


ft_model_baseline = Sequential()

ft_model_baseline.add(Embedding(vocab_size, embedding_size, \
                    input_length=maxlen))

ft_model_baseline.add(SimpleRNN(64, return_sequences = True))
ft_model_baseline.add(SimpleRNN(64, return_sequences = True))
ft_model_baseline.add(Dense(vocab_size, activation='softmax'))


ft_model_baseline.load_weights('model_baseline_weights.h5')
ft_model_baseline.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ft_model_baseline.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 53, 10)            61900     
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 53, 64)            4800      
_________________________________________________________________
simple_rnn_11 (SimpleRNN)    (None, 53, 64)            8256      
_________________________________________________________________
dense_5 (Dense)              (None, 53, 6190)          402350    
Total params: 477,306
Trainable params: 477,306
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)

In [ ]:
ft_model_baseline.fit(X_pad_ft, np.array(Y_pad_ft_m1), epochs=100,  callbacks = callback )

Epoch 1/100
2/2 [==============================] - 2s 38ms/step - loss: 3.6055 - accuracy: 0.7595
Epoch 2/100
2/2 [==============================] - 0s 37ms/step - loss: 3.4183 - accuracy: 0.7584
Epoch 3/100
2/2 [==============================] - 0s 36ms/step - loss: 3.2386 - accuracy: 0.7572
Epoch 4/100
2/2 [==============================] - 0s 35ms/step - loss: 3.0671 - accuracy: 0.7555
Epoch 5/100
2/2 [==============================] - 0s 33ms/step - loss: 2.9459 - accuracy: 0.7531
Epoch 6/100
2/2 [==============================] - 0s 33ms/step - loss: 2.8072 - accuracy: 0.7536
Epoch 7/100
2/2 [==============================] - 0s 32ms/step - loss: 2.7249 - accuracy: 0.7532
Epoch 8/100
2/2 [==============================] - 0s 32ms/step - loss: 2.6616 - accuracy: 0.7456
Epoch 9/100
2/2 [==============================] - 0s 32ms/step - loss: 2.5903 - accuracy: 0.7432
Epoch 10/100
2/2 [==============================] - 0s 32ms/step - loss: 2.5684 - accuracy: 0.7299
Epoch 11/100
2/2 [=

## Test Predictions

In [ ]:
df_test = pd.read_csv("/Users/shivanibalaji/Downloads/test-2.csv")
df_test.head()

,Questions
0,Will the pre-class session be recorded?
1,What is the deadline for quiz submission?
2,What is the deadline for exercise submission?
3,How many hours do I need to complete this course?
4,Who will grade the exercise?


In [ ]:
pred_test = []
for q in df_test.Questions.values:
    pred_test.append(predict(q, ft_model_baseline))

In [ ]:
predictions_test_data = pd.DataFrame()
predictions_test_data['Questions'] = df_test.Questions.values
predictions_test_data['Predicted_answers'] = pred_test

In [ ]:
predictions_test_data

,Questions,Predicted_answers
0,Will the pre-class session be recorded?,no your week will be the </s>
1,What is the deadline for quiz submission?,unk official pythons be
2,What is the deadline for exercise submission?,unk official pythons you will be be be be
3,How many hours do I need to complete this course?,unk unk the you
4,Who will grade the exercise?,the unk
5,Why is the auto-grader failing me?,no a and you teams are <s> 4 the the the be </s>
6,Do I do the exercises individually?,not your the be your be be be be
7,Is the lab compulsory?,all will you will be be week
8,Will the sessions be recorded?,no
9,Can I have access to the recorded videos?,does <s> video the the be is be be be be be be be


# Seq 2 Seq Model


In [ ]:
clean_question_train = clean_data(questions_train)[:10000]
clean_answer_train = clean_data(answers_train)[:10000]

In [ ]:
clean_question_ft = clean_data(questions_ft)
clean_answer_ft = clean_data(answers_ft)

In [ ]:
target_regex = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n\'0123456789'

tokenizer = Tokenizer(filters=target_regex)
tokenizer.fit_on_texts(clean_question_train + clean_answer_train +clean_question_ft +  clean_answer_ft)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print('Vocabulary size : {}'.format(VOCAB_SIZE))

Vocabulary size : 12497


In [ ]:
tokenized_questions_train = tokenizer.texts_to_sequences(clean_question_train)

maxlen_questions_train = max([len(x) for x in tokenized_questions_train])
encoder_input_data_train = pad_sequences(tokenized_questions_train,
                                   maxlen=maxlen_questions_train,
                                   padding='post')

print(encoder_input_data_train.shape)

(10000, 30)


In [ ]:
tokenized_answers_train = tokenizer.texts_to_sequences(clean_answer_train)

maxlen_answers_train = max([len(x) for x in tokenized_answers_train])
decoder_input_data_train = pad_sequences(tokenized_answers_train,
                                   maxlen=maxlen_answers_train,
                                   padding='post')
print(decoder_input_data_train.shape)

(10000, 43)


In [ ]:
for i in range(len(tokenized_answers_train)):
    tokenized_answers_train[i] = tokenized_answers_train[i][1:]
padded_answers_train = pad_sequences(tokenized_answers_train, maxlen=maxlen_answers_train, padding='post')
decoder_output_data_train = to_categorical(padded_answers_train, VOCAB_SIZE)

print(decoder_output_data_train.shape)

(10000, 43, 12497)


In [ ]:
'''

enc_inputs = Input(shape=(None,))
enc_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(enc_inputs)
_, state_h, state_c = LSTM(200, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(dec_inputs)
dec_lstm = LSTM(200, return_state=True, return_sequences=True)
dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)
dec_dense = Dense(VOCAB_SIZE, activation=softmax)
output = dec_dense(dec_outputs)

model = Model([enc_inputs, dec_inputs], output)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')

model.summary()

callback = EarlyStopping(monitor='loss', patience=3)

model.fit([encoder_input_data_train, decoder_input_data_train],
          decoder_output_data_train,
          batch_size=50,
          epochs=100, 
          callbacks = callback, 
          validation_split = 0.2)
model.save('model_seq2seq.h5')

''' 

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, None, 200)    2499400     input_26[0][0]                   
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, None, 200)    2499400     input_27[0][0]                   
___________________________________________________________________________________________

In [ ]:
#model.save_weights('model_seq2seq_new.h5')

In [ ]:

def make_inference_models():
    dec_state_input_h = Input(shape=(200,))
    dec_state_input_c = Input(shape=(200,))
    dec_states_inputs = [dec_state_input_h, dec_state_input_c]
    dec_outputs, state_h, state_c = dec_lstm(dec_embedding,
                                             initial_state=dec_states_inputs)
    dec_states = [state_h, state_c]
    dec_outputs = dec_dense(dec_outputs)
    dec_model = Model(
        inputs=[dec_inputs] + dec_states_inputs,
        outputs=[dec_outputs] + dec_states)
    print('Inference decoder:')
    dec_model.summary()
    print('Inference encoder:')
    enc_model = Model(inputs=enc_inputs, outputs=enc_states)
    enc_model.summary()
    return enc_model, dec_model


In [ ]:
def str_to_tokens(sentence: str):
    words = sentence.lower().split()
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word, '')
        if result != '':
            tokens_list.append(result)
    return pad_sequences([tokens_list],
                         maxlen=maxlen_questions,
                         padding='post')

In [ ]:
#enc_model, dec_model = make_inference_models()

Inference decoder:
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, None, 200)    2499400     input_27[0][0]                   
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_29 (InputLayer)           [(None, 200)]        0                                            
________________________________________________________________________

In [ ]:
def decode_sequnce(input_sentence):
    states_values = enc_model.predict(
        str_to_tokens(input_sentence))
    
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]
                                              + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != 'end':
                    decoded_translation += ' {}'.format(word)
                sampled_word = word

        if sampled_word == 'end' \
                or len(decoded_translation.split()) \
                > maxlen_answers:
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]

        
        return decoded_translation

In [ ]:
decode_sequnce('Where is Beyonce')

' in origin royal in solar power royal royal a tibetan power and power royal power and and royal piano royal power plant royal in music royal royal canadian congo royal heating royal royal and royal canadian air and pure royal royal royal and or and small'

## Fine Tune Model

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
print('Vocabulary size : {}'.format(VOCAB_SIZE_ft))

Vocabulary size : 241


In [ ]:
tokenized_questions_ft = tokenizer.texts_to_sequences(clean_question_ft)

maxlen_questions_ft = max([len(x) for x in tokenized_questions_ft])
encoder_input_data_ft = pad_sequences(tokenized_questions_ft,
                                   maxlen=maxlen_questions_ft,
                                   padding='post')

print(encoder_input_data_ft.shape)

(33, 15)


In [ ]:
maxlen_questions_ft

15

In [ ]:
maxlen_answers_ft

51

In [ ]:
tokenized_answers_ft = tokenizer.texts_to_sequences(clean_answer_ft)

maxlen_answers_ft = max([len(x) for x in tokenized_answers_ft])
decoder_input_data_ft = pad_sequences(tokenized_answers_ft,
                                   maxlen=maxlen_answers_ft,
                                   padding='post')
print(decoder_input_data_ft.shape)



(33, 51)


In [ ]:
for i in range(len(tokenized_answers_ft)):
    tokenized_answers_ft[i] = tokenized_answers_ft[i][1:]
padded_answers_ft = pad_sequences(tokenized_answers_ft, maxlen=maxlen_answers_ft, padding='post')
decoder_output_data_ft = to_categorical(padded_answers_ft, VOCAB_SIZE)

print(decoder_output_data_ft.shape)

(33, 51, 12497)


In [ ]:
enc_inputs = Input(shape=(None,))
enc_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(enc_inputs)
_, state_h, state_c = LSTM(200, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(dec_inputs)
dec_lstm = LSTM(200, return_state=True, return_sequences=True)
dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)
dec_dense = Dense(VOCAB_SIZE, activation=softmax)
output = dec_dense(dec_outputs)

model_ft = Model([enc_inputs, dec_inputs], output)

In [ ]:
model_ft.load_weights('model_seq2seq_new.h5')

In [ ]:
model_ft.compile(optimizer=RMSprop(), loss='categorical_crossentropy')

In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)

In [ ]:
model_ft.fit([encoder_input_data_ft, decoder_input_data_ft],decoder_output_data_ft,batch_size=50,epochs=100, callbacks = callback, validation_split = 0.2)

In [ ]:
def make_inference_models():
    dec_state_input_h = Input(shape=(200,))
    dec_state_input_c = Input(shape=(200,))
    dec_states_inputs = [dec_state_input_h, dec_state_input_c]
    dec_outputs, state_h, state_c = dec_lstm(dec_embedding,
                                             initial_state=dec_states_inputs)
    dec_states = [state_h, state_c]
    dec_outputs = dec_dense(dec_outputs)
    dec_model = Model(
        inputs=[dec_inputs] + dec_states_inputs,
        outputs=[dec_outputs] + dec_states)
    print('Inference decoder:')
    dec_model.summary()
    print('Inference encoder:')
    enc_model = Model(inputs=enc_inputs, outputs=enc_states)
    enc_model.summary()
    return enc_model, dec_model


In [ ]:
enc_model, dec_model = make_inference_models()

Inference decoder:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    2499400     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 200)]        0                                            
_________________________________________________________________________

In [ ]:
def str_to_tokens(sentence: str):
    words = sentence.lower().split()
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word, '')
        if result != '':
            tokens_list.append(result)
    return pad_sequences([tokens_list],
                         maxlen=maxlen_questions_ft,
                         padding='post')

In [ ]:
def decode_sequence(input_sentence):
    states_values = enc_model.predict(str_to_tokens(input_sentence))
    
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]
                                              + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != 'end':
                    decoded_translation += ' {}'.format(word)
                sampled_word = word

        if sampled_word == 'end' \
                or len(decoded_translation.split()) \
                > maxlen_answers_ft:
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]

        
    return decoded_translation

## Test Preditions

In [ ]:
df_test = pd.read_csv("/Users/shivanibalaji/Downloads/test-2.csv")
df_test.head()

,Questions
0,Will the pre-class session be recorded?
1,What is the deadline for quiz submission?
2,What is the deadline for exercise submission?
3,How many hours do I need to complete this course?
4,Who will grade the exercise?


In [ ]:
pred_test = []
for q in df_test.Questions.values:
    pred_test.append(decode_sequence(q))

In [ ]:
predictions_test_data = pd.DataFrame()
predictions_test_data['Questions'] = df_test.Questions.values
predictions_test_data['Predicted_answers'] = pred_test

In [ ]:
predictions_test_data

,Questions,Predicted_answers
0,Will the pre-class session be recorded?,and part for the following the of the followi...
1,What is the deadline for quiz submission?,in special each and the force royal of the to...
2,What is the deadline for exercise submission?,of the canadian army army royal the world to ...
3,How many hours do I need to complete this course?,of the water of naraka royal a music awards r...
4,Who will grade the exercise?,and are are royal are and all all the with an...
5,Why is the auto-grader failing me?,are is be royal is material royal in this roy...
6,Do I do the exercises individually?,for a each of each royal and the heat royal r...
7,Is the lab compulsory?,of the united provinces school royal the be k...
8,Will the sessions be recorded?,and are is will royal and understanding royal...
9,Can I have access to the recorded videos?,all of the canadian forces royal each of the ...
